In [1]:
# -*- coding: UTF-8 -*-    
import pandas as pd
from pandas.io.json import json_normalize
import json
import pathlib

In [2]:
# 读入数据
def read_finmain():
    p = pathlib.Path('./json')
    files = list(p.glob('*.json'))

    dframes = []
    for file in files:
        with file.open() as f:
            data_str = f.read()
            data_list = json.loads(data_str)
            df = json_normalize(data_list["list"])
            df['symbol'] = file.stem.lower()
            dframes.append(df)

    r = pd.concat(dframes)
    r.reportdate = pd.to_datetime(r['reportdate'])
    r.reportdate = r.reportdate.dt.to_period("Q")
    r.to_csv('account.csv', index=False)

In [3]:
def read_fzb():
    p = pathlib.Path('./csv')
    files = list(p.glob('*_fzb.csv'))

    dframes = []
    for file in files:
        df = pd.read_csv(str(file))
        df['symbol'] = file.stem.lower()[:8]
        dframes.append(df)

    r = pd.concat(dframes)
    r['报表日期'] = pd.to_datetime(r['报表日期'], format='%Y%m%d')
    r['报表日期'] = r['报表日期'].dt.to_period("Q")
    return r

d = read_fzb()
start = pd.Period('2017Q3')
dnew = d.loc[d['报表日期'] >= start ,['symbol', '报表日期', '实收资本(或股本)']]
dnew.set_index('symbol', inplace=True)
dnew.rename(columns={'报表日期': 'reportdate'}, inplace=True)
dnew.to_csv('totalshare.csv')
dnew.head()

,reportdate,实收资本(或股本)
symbol,,
sz300004,2017Q3,509218928.0
sz000893,2017Q3,756903272.0
sz002612,2017Q3,400000000.0
sh600805,2017Q3,850894494.0
sz300014,2017Q3,855985593.0
